In [135]:
pip install import-ipynb


Note: you may need to restart the kernel to use updated packages.


In [136]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [137]:
import import_ipynb
import Cleaning_LoanPrediction as clp

In [138]:
df_new = clp.df_new
test = clp.test
df_new.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome,TotalIncome_log,Type
0,LP001002,1,No,0,0,No,5849,0.0,146.412162,360.0,1,Urban,1,5849.0,8.674026,Train
1,LP001003,1,Yes,1,0,No,4583,1508.0,128.000000,360.0,1,Rural,0,6091.0,8.714568,Train
2,LP001005,1,Yes,0,0,Yes,3000,0.0,66.000000,360.0,1,Urban,1,3000.0,8.006368,Train
3,LP001006,1,Yes,0,1,No,2583,2358.0,120.000000,360.0,1,Urban,1,4941.0,8.505323,Train
4,LP001008,1,No,0,0,No,6000,0.0,141.000000,360.0,1,Urban,1,6000.0,8.699515,Train


In [139]:
#Create a flag for Train and Test Data set
df_new['Type']='Train' 
test['Type']='Test'

C:\Users\ZAQ\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [140]:
fulldata = pd.concat([df_new , test],axis=0,sort=True)
fulldata=fulldata.drop(['TotalIncome','TotalIncome_log'], axis=1) #we drop these two columns because in model building and in predictions , we don't need these columns.
fulldata.tail()

,ApplicantIncome,CoapplicantIncome,Credit_History,Dependents,Education,Gender,LoanAmount,Loan_Amount_Term,Loan_ID,Loan_Status,Married,Property_Area,Self_Employed,Type
362,4009,1777.0,1.0,3+,Not Graduate,Male,113.0,360.0,LP002971,NaN,Yes,Urban,Yes,Test
363,4158,709.0,1.0,0,Graduate,Male,115.0,360.0,LP002975,NaN,Yes,Urban,No,Test
364,3250,1993.0,NaN,0,Graduate,Male,126.0,360.0,LP002980,NaN,No,Semiurban,No,Test
365,5000,2393.0,1.0,0,Graduate,Male,158.0,360.0,LP002986,NaN,Yes,Rural,No,Test
366,9200,0.0,1.0,0,Graduate,Male,98.0,180.0,LP002989,NaN,No,Rural,Yes,Test


In [141]:
fulldata.isnull().sum()

ApplicantIncome        0
CoapplicantIncome      0
Credit_History        29
Dependents            10
Education              0
Gender                11
LoanAmount             5
Loan_Amount_Term       6
Loan_ID                0
Loan_Status          367
Married                0
Property_Area          0
Self_Employed         23
Type                   0
dtype: int64

In [142]:
#Identify categorical and continuous variables
ID_col = ['Loan_ID']
target_col = ["Loan_Status"]
cat_cols = ['Credit_History','Dependents','Gender','Married','Education','Property_Area','Self_Employed']

In [143]:
#Imputing Missing values with mean for continuous variable
fulldata['LoanAmount'].replace(to_replace=np.nan,value=fulldata['LoanAmount'].mean(),inplace=True)
fulldata['ApplicantIncome'].replace(to_replace=np.nan,value=fulldata['ApplicantIncome'].mean(),inplace=True)
fulldata['CoapplicantIncome'].replace(to_replace=np.nan,value=fulldata['CoapplicantIncome'].mean(),inplace=True)
fulldata['Loan_Amount_Term'].replace(to_replace=np.nan,value=fulldata['Loan_Amount_Term'].mean(),inplace=True)

In [144]:
#Imputing missing values with mode for categorical variable

#fill missing value in Gender
fulldata['Gender'].replace(to_replace=np.nan,value=fulldata['Gender'].mode()[0],inplace=True)

#fill missing value in Married
fulldata['Married'].replace(to_replace=np.nan,value=fulldata['Married'].mode()[0],inplace=True)

#fill missing value in Dependents
fulldata['Dependents'].replace(to_replace=np.nan,value=fulldata['Dependents'].mode()[0],inplace=True)

#fill missing value in Credit_History
fulldata['Credit_History'].replace(to_replace=np.nan,value=fulldata['Credit_History'].mode()[0],inplace=True)

#fill missing value in Self Employed
fulldata['Self_Employed'].replace(to_replace=np.nan,value=fulldata['Self_Employed'].mode()[0],inplace=True)


In [145]:
for var in cat_cols:
    number = LabelEncoder()
    fulldata[var] = number.fit_transform(fulldata[var].astype('str'))

In [146]:
fulldata.tail()
#education : 3--non-graduated
#gender : 3--male

,ApplicantIncome,CoapplicantIncome,Credit_History,Dependents,Education,Gender,LoanAmount,Loan_Amount_Term,Loan_ID,Loan_Status,Married,Property_Area,Self_Employed,Type
362,4009,1777.0,1,3,3,3,113.0,360.0,LP002971,NaN,1,2,1,Test
363,4158,709.0,1,0,2,3,115.0,360.0,LP002975,NaN,1,2,0,Test
364,3250,1993.0,1,0,2,3,126.0,360.0,LP002980,NaN,0,1,0,Test
365,5000,2393.0,1,0,2,3,158.0,360.0,LP002986,NaN,1,0,0,Test
366,9200,0.0,1,0,2,3,98.0,180.0,LP002989,NaN,0,0,1,Test


In [147]:
#The chances of getting a loan will be higher for:
    #Applicants having a credit history 
    #Applicants with higher applicant and co-applicant incomes.
    #Applicants with higher education level.
    #Properties in urban areas with high growth perspectives.
#So let’s make our model with ‘Credit_History’, ‘Education’ & ‘Gender'

In [148]:
train_modified=fulldata[fulldata['Type']=='Train']
train_modified["Loan_Status"] = number.fit_transform(train_modified["Loan_Status"].astype('str'))
test_modified=fulldata[fulldata['Type']=='Test']

C:\Users\ZAQ\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [149]:
predictors=['Credit_History','Education','Gender'] #predictors refers to independent variable
x_train =  train_modified[list(predictors)].values
y_train =  train_modified["Loan_Status"].values
x_test  =  test_modified[list(predictors)].values

x_test.shape
# print(x_train.shape)

(367, 3)

In [150]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [151]:
knnclassifier = KNeighborsClassifier(n_neighbors=5)

#fit the model
knnclassifier.fit(x_train,y_train)
#predict on training set
y_pred = knnclassifier.predict(x_train)
#print accuracy
accuracy = metrics.accuracy_score(y_pred,y_train)
print ("Accuracy : %s" % "{0:.3%}".format(accuracy))

Accuracy : 81.802%


In [157]:
model = LogisticRegression()

#fit the model
model.fit(x_train,y_train)

#predict the training set
y_pred = model.predict(x_test)
y_pred = number.inverse_transform(y_pred)
test_modified['Loan_Status'] = y_pred

C:\Users\ZAQ\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ZAQ\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [153]:
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression()

# evaluate model
scores = cross_val_score(model, x_test, y_pred, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (scores.mean(), scores.std()))

#Fit the model again so that it can be refered outside the function:
model.fit(df_new[predictors],df_new['Loan_Status'])

Accuracy: 1.000 (0.000)


C:\Users\ZAQ\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [155]:
test_modified.tail(15)
# in education == 2 means graduated 
# in gender == 3 means male

,ApplicantIncome,CoapplicantIncome,Credit_History,Dependents,Education,Gender,LoanAmount,Loan_Amount_Term,Loan_ID,Loan_Status,Married,Property_Area,Self_Employed,Type
352,5817,910.0,1,0,2,3,109.0,360.0,LP002907,1.0,1,2,0,Test
353,5119,3769.0,1,0,2,3,120.0,360.0,LP002920,1.0,1,0,0,Test
354,5316,187.0,0,3,3,3,158.0,180.0,LP002921,0.0,1,1,0,Test
355,7603,1213.0,1,3,2,3,197.0,360.0,LP002932,1.0,1,2,0,Test
356,3791,1936.0,1,1,2,3,85.0,360.0,LP002935,1.0,1,2,0,Test
357,2500,0.0,1,0,2,3,60.0,360.0,LP002952,1.0,0,2,0,Test
358,3132,0.0,1,2,3,3,76.0,360.0,LP002954,1.0,1,0,0,Test
359,4000,2667.0,1,0,2,3,152.0,360.0,LP002962,1.0,0,1,0,Test
360,8550,4255.0,1,0,2,2,96.0,360.0,LP002965,1.0,1,2,0,Test
361,2269,2167.0,1,1,2,3,99.0,360.0,LP002969,1.0,1,1,0,Test


In [158]:
#completed the prediction of loan status in test data set 